# 9章 素性ベースの文法の構築

自然言語には，8章で説明したような単純な方法では扱いが難しい文法的構造が広範囲にわたって存在する．そこで柔軟性を確保するために，SやNP，Vといった文法範疇の扱いを変えることにする．アトミックなラベルの代わりに，それらを辞書のような構造に分解し，素性が様々な値を取ることができるようにするのだ．

## 本章の目的
1. 文脈自由文法の枠組みを拡張し，文法範疇と生成規則をより詳細に制御するにはどうすればよいか．

2. 素性構造の形式特性は主に何で，どのように計算によって扱えばよいか．

3. 素性ベースの文法を用いると，どのような種類の言語パターンや文法生成規則を扱うことができるか．

# 9.1 文法素性

In [2]:
# -*- coding:utf-8 -*-

kim = {'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chase = {'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase'}

# sub(主語)とobj(目的語)を動詞が文法項と結合したときに埋められる
# プレースホルダとして使う．

chase['AGT'] = 'sbj'
chase['PAT'] = 'obj'

In [8]:
# -*- coding:utf-8 -*-

sent = "Kim chased Lee"
tokens = sent.split()
lee = {'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}

# fsの各要素のorthographyが引数と一致すればそれを返す

def lex2fs(word):
    for fs in [kim, lee, chase]:
        if fs['ORTH'] == word:
            print(fs['ORTH'], word)
            print(fs)
            return fs
        
# 3つの変数に関数の返り値を代入

subj, verb, obj = lex2fs(tokens[0]), lex2fs(tokens[1]), lex2fs(tokens[2])

verb['AGT'] = subj['REF']     # agent of 'chase' is Kim
verb['PAT'] = obj['REF']      # patient of 'chase' is Lee

# check featstruct of 'chase'
for k in ['ORTH', 'REL', 'AGT', 'PAT']:
    print("%-5s => %s" % (k, verb[k]))

Kim Kim
{'CAT': 'NP', 'ORTH': 'Kim', 'REF': 'k'}
chased chased
{'CAT': 'V', 'ORTH': 'chased', 'REL': 'chase', 'AGT': 'sbj', 'PAT': 'obj'}
Lee Lee
{'CAT': 'NP', 'ORTH': 'Lee', 'REF': 'l'}
ORTH  => chased
REL   => chase
AGT   => k
PAT   => l


In [10]:
import nltk

nltk.data.show_cfg('grammars/book_grammars/feat0.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
# S expansion productions
S -> NP[NUM=?n] VP[NUM=?n]
# NP expansion productions
NP[NUM=?n] -> N[NUM=?n] 
NP[NUM=?n] -> PropN[NUM=?n] 
NP[NUM=?n] -> Det[NUM=?n] N[NUM=?n]
NP[NUM=pl] -> N[NUM=pl] 
# VP expansion productions
VP[TENSE=?t, NUM=?n] -> IV[TENSE=?t, NUM=?n]
VP[TENSE=?t, NUM=?n] -> TV[TENSE=?t, NUM=?n] NP
# ###################
# Lexical Productions
# ###################
Det[NUM=sg] -> 'this' | 'every'
Det[NUM=pl] -> 'these' | 'all'
Det -> 'the' | 'some' | 'several'
PropN[NUM=sg]-> 'Kim' | 'Jody'
N[NUM=sg] -> 'dog' | 'girl' | 'car' | 'child'
N[NUM=pl] -> 'dogs' | 'girls' | 'cars' | 'children' 
IV[TENSE=pres,  NUM=sg] -> 'disappears' | 'walks'
TV[TENSE=pres, NUM=sg] -> 'sees' | 'likes'
IV[TENSE=pres,  NUM=pl] -> 'disappear' | 'walk'
TV[TENSE=pres, NUM=pl] -> 'see' | 'like'
IV[TENSE=past] -> 'disappeared' | 'walked'
TV[TENSE=past] -> 'saw' | 'liked'


In [18]:
tokens = 'Kim likes children'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat0.fcfg', trace=2)
for tree in cp.parse(tokens):
    print(tree)

|.Kim .like.chil.|
Leaf Init Rule:
|[----]    .    .| [0:1] 'Kim'
|.    [----]    .| [1:2] 'likes'
|.    .    [----]| [2:3] 'children'
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] PropN[NUM='sg'] -> 'Kim' *
Feature Bottom Up Predict Combine Rule:
|[----]    .    .| [0:1] NP[NUM='sg'] -> PropN[NUM='sg'] *
Feature Bottom Up Predict Combine Rule:
|[---->    .    .| [0:1] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'sg'}
Feature Bottom Up Predict Combine Rule:
|.    [----]    .| [1:2] TV[NUM='sg', TENSE='pres'] -> 'likes' *
Feature Bottom Up Predict Combine Rule:
|.    [---->    .| [1:2] VP[NUM=?n, TENSE=?t] -> TV[NUM=?n, TENSE=?t] * NP[] {?n: 'sg', ?t: 'pres'}
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] N[NUM='pl'] -> 'children' *
Feature Bottom Up Predict Combine Rule:
|.    .    [----]| [2:3] NP[NUM='pl'] -> N[NUM='pl'] *
Feature Bottom Up Predict Combine Rule:
|.    .    [---->| [2:3] S[] -> NP[NUM=?n] * VP[NUM=?n] {?n: 'pl'}
Feature Single Edge Fundame

# 9.2 素性構造の処理

本節ではNLTKで素性構造をどのように構築，操作するかについて解説する．さらに２つの異なる素性構造に含まれる情報を統合する方法である，単一化という基本操作についてみていく．NLTKではFeatStruct()コンストラクタを用いて素性構造を宣言できる．アトミックな素性値としては，文字列もしくは整数を用いることができる．

In [23]:
# -*- coding:utf-8 -*-

fs1 = nltk.FeatStruct(TENSE='past', NUM='sg')
print(fs1)

# 基本的な操作は辞書と同じ

fs1 = nltk.FeatStruct(PER=3, NUM='p1', GND='fem')
print(fs1['GND'])
fs1['CASE'] = 'acc'
print(fs1)

# 複合値を持つ素性構造も定義できる

fs2 = nltk.FeatStruct(POS='N', AGR=fs1)
print(fs2)
print(fs2['AGR']['PER'])

[ NUM   = 'sg'   ]
[ TENSE = 'past' ]
fem
[ CASE = 'acc' ]
[ GND  = 'fem' ]
[ NUM  = 'p1'  ]
[ PER  = 3     ]
[       [ CASE = 'acc' ] ]
[ AGR = [ GND  = 'fem' ] ]
[       [ NUM  = 'p1'  ] ]
[       [ PER  = 3     ] ]
[                        ]
[ POS = 'N'              ]
3


In [24]:
# 素性構造を定義するほかの方法として，素性と値のペアを「素性＝値」の形式で
# 表現したカッコつき文字列を用いることもできる

print(nltk.FeatStruct("[POS='N', AGR=[PER=3, NUM='p1', GND='fem']]"))

[       [ GND = 'fem' ] ]
[ AGR = [ NUM = 'p1'  ] ]
[       [ PER = 3     ] ]
[                       ]
[ POS = 'N'             ]


In [25]:
# 素性構造を用いて，人物に関する情報を符号化することもできる

print(nltk.FeatStruct(name='Lee', telno='01 27 86 42 96', age=33))

[ age   = 33               ]
[ name  = 'Lee'            ]
[ telno = '01 27 86 42 96' ]


素性構造をグラフ，具体的には無閉路有効グラフ（DAG）として表現すると便利なことがある．

再入を行列的表現によって表示するには，共有素性構造が最初に出現するときに(1)のように括弧つき整数によって接頭辞をつける．この構造を以降参照する場合にはすべて->(1)という表記を用いる．

In [27]:
# -*- coding:utf-8 -*-

print(nltk.FeatStruct("""[NAME='Lee',
    ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
    SPOUSE=[NAME='Kim', ADDRESS->(1)]]"""))

# 単一の素性構造にどれだけタグを含めてもよい
print('--------------------------------------')
print(nltk.FeatStruct("[A='a', B=(1)[C='c'], D->(1), E->(1)]"))

[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]
--------------------------------------
[ A = 'a'             ]
[                     ]
[ B = (1) [ C = 'c' ] ]
[                     ]
[ D -> (1)            ]
[ E -> (1)            ]


In [29]:
# 2つの素性構造の情報を合併する操作は単一化と呼ばれ，unify()メソッドを利用

fs1 = nltk.FeatStruct(NUMBER=74, STREET='rue Pascal')
fs2 = nltk.FeatStruct(CITY='Paris')
print(fs1.unify(fs2))

# 単一化は対照的である

print('------------------------------------------------------')
print(fs2.unify(fs1))

[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]
------------------------------------------------------
[ CITY   = 'Paris'      ]
[ NUMBER = 74           ]
[ STREET = 'rue Pascal' ]


In [30]:
# 単一化の失敗

fs0 = nltk.FeatStruct(A='a')
fs1 = nltk.FeatStruct(A='b')
fs2 = fs0.unify(fs1)
print(fs2)

None


In [31]:
fs0 = nltk.FeatStruct("""[NAME=Lee,
                              ADDRESS=[NUMBER=74,
                              STREET='rue Pascal'],
                          SPOUSE= [NAME=Kim,
                              ADDRESS=[NUMBER=74,
                              STREET='rue Pascal']]]""")

print(fs0)

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


In [32]:
# Kimの住所に対してCITYを定義して拡張する

fs1 = nltk.FeatStruct("[SPOUSE = [ADDRESS = [CITY = PARIS]]]")
print(fs1.unify(fs0))

[ ADDRESS = [ NUMBER = 74           ]               ]
[           [ STREET = 'rue Pascal' ]               ]
[                                                   ]
[ NAME    = 'Lee'                                   ]
[                                                   ]
[           [           [ CITY   = 'PARIS'      ] ] ]
[           [ ADDRESS = [ NUMBER = 74           ] ] ]
[ SPOUSE  = [           [ STREET = 'rue Pascal' ] ] ]
[           [                                     ] ]
[           [ NAME    = 'Kim'                     ] ]


In [33]:
# 素性共有版のfs2と単一化
fs2 = nltk.FeatStruct("""[NAME=Lee,
                      ADDRESS=(1)[NUMBER=74, STREET='rue Pascal'],
                      SPOUSE=[NAME=Kim, ADDRESS->(1)]]""")

print(fs1.unify(fs2))

[               [ CITY   = 'PARIS'      ] ]
[ ADDRESS = (1) [ NUMBER = 74           ] ]
[               [ STREET = 'rue Pascal' ] ]
[                                         ]
[ NAME    = 'Lee'                         ]
[                                         ]
[ SPOUSE  = [ ADDRESS -> (1)  ]           ]
[           [ NAME    = 'Kim' ]           ]


# 9.3 素性ベースの文法の拡張

いかに示す文法は，空所を導入する生成規則を含んでいる．空所素性を正しく買いに伝播させるためには
空所を変数値とともにS,VP,NPを展開する生成規則の両辺に追加する必要がある．

In [39]:
# -*- coding:utf-8 -*-

# 転置節および空所範疇を利用した長距離依存に対応する生成規則を含む文法

nltk.data.show_cfg('grammars/book_grammars/feat1.fcfg')

% start S
# ###################
# Grammar Productions
# ###################
S[-INV] -> NP VP
S[-INV]/?x -> NP VP/?x
S[-INV] -> NP S/NP
S[-INV] -> Adv[+NEG] S[+INV]
S[+INV] -> V[+AUX] NP VP
S[+INV]/?x -> V[+AUX] NP VP/?x
SBar -> Comp S[-INV]
SBar/?x -> Comp S[-INV]/?x
VP -> V[SUBCAT=intrans, -AUX]
VP -> V[SUBCAT=trans, -AUX] NP
VP/?x -> V[SUBCAT=trans, -AUX] NP/?x
VP -> V[SUBCAT=clause, -AUX] SBar
VP/?x -> V[SUBCAT=clause, -AUX] SBar/?x
VP -> V[+AUX] VP
VP/?x -> V[+AUX] VP/?x
# ###################
# Lexical Productions
# ###################
V[SUBCAT=intrans, -AUX] -> 'walk' | 'sing'
V[SUBCAT=trans, -AUX] -> 'see' | 'like'
V[SUBCAT=clause, -AUX] -> 'say' | 'claim'
V[+AUX] -> 'do' | 'can'
NP[-WH] -> 'you' | 'cats'
NP[+WH] -> 'who'
Adv[+NEG] -> 'rarely' | 'never'
NP/NP ->
Comp -> 'that'


In [40]:
# who do you claim that you like を構文解析する

tokens = 'who do you claim that you like'.split()
from nltk import load_parser
cp = load_parser('grammars/book_grammars/feat1.fcfg')
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[+WH] who)
  (S[+INV]/NP[]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[]/NP[]
      (V[-AUX, SUBCAT='clause'] claim)
      (SBar[]/NP[]
        (Comp[] that)
        (S[-INV]/NP[]
          (NP[-WH] you)
          (VP[]/NP[] (V[-AUX, SUBCAT='trans'] like) (NP[]/NP[] )))))))


In [41]:
# 空所のない文の構文解析

tokens = 'you claim that you like cats'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (NP[-WH] you)
  (VP[]
    (V[-AUX, SUBCAT='clause'] claim)
    (SBar[]
      (Comp[] that)
      (S[-INV]
        (NP[-WH] you)
        (VP[] (V[-AUX, SUBCAT='trans'] like) (NP[-WH] cats))))))


In [42]:
# wh 構造を持たない転置文を受け取る

tokens = 'rarely do you sing'.split()
for tree in cp.parse(tokens):
    print(tree)

(S[-INV]
  (Adv[+NEG] rarely)
  (S[+INV]
    (V[+AUX] do)
    (NP[-WH] you)
    (VP[] (V[-AUX, SUBCAT='intrans'] sing))))


In [43]:
nltk.data.show_cfg('grammars/book_grammars/german.fcfg')

% start S
# Grammar Productions
S -> NP[CASE=nom, AGR=?a] VP[AGR=?a]
NP[CASE=?c, AGR=?a] -> PRO[CASE=?c, AGR=?a]
NP[CASE=?c, AGR=?a] -> Det[CASE=?c, AGR=?a] N[CASE=?c, AGR=?a]
VP[AGR=?a] -> IV[AGR=?a]
VP[AGR=?a] -> TV[OBJCASE=?c, AGR=?a] NP[CASE=?c]
# Lexical Productions
# Singular determiners
# masc
Det[CASE=nom, AGR=[GND=masc,PER=3,NUM=sg]] -> 'der' 
Det[CASE=dat, AGR=[GND=masc,PER=3,NUM=sg]] -> 'dem'
Det[CASE=acc, AGR=[GND=masc,PER=3,NUM=sg]] -> 'den'
# fem
Det[CASE=nom, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
Det[CASE=dat, AGR=[GND=fem,PER=3,NUM=sg]] -> 'der'
Det[CASE=acc, AGR=[GND=fem,PER=3,NUM=sg]] -> 'die' 
# Plural determiners
Det[CASE=nom, AGR=[PER=3,NUM=pl]] -> 'die' 
Det[CASE=dat, AGR=[PER=3,NUM=pl]] -> 'den' 
Det[CASE=acc, AGR=[PER=3,NUM=pl]] -> 'die' 
# Nouns
N[AGR=[GND=masc,PER=3,NUM=sg]] -> 'Hund'
N[CASE=nom, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[CASE=dat, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunden'
N[CASE=acc, AGR=[GND=masc,PER=3,NUM=pl]] -> 'Hunde'
N[AGR=[GND=fem,PER=3,

# 9.4 まとめ

文脈自由文法で従来使われてきている範疇はアトミックな記号である．素性構造を用いる重要な動機として，より粒度の細かな識別に対応できることが挙げられる．ほかの方法ではアトミックな範疇を大幅に増やさなければならない．

素性値に変数を用いることで文法生成規則における制約を表現することができ，異なる素性の指定を相互に依存させることができる．

通常は語彙レベルには固定値の素性を割り当て，句の素性値を制限し，子の対応する値と単一化する．
素性値はアトミック値もしくは複合型である．アトミック値の１つの特殊な場合としてブール値があり，これは慣例的に[+/- 素性]によって表される．

素性同士が値（アトミック値，複合型を問わず）を共有することもできる，共通の値を持つ構造は歳入と呼ばれ，AVMにおいて共通の値は添え字（もしきはタグ）によって表現される．

素性構造におけるパスは，グラフ表現の根から始まる弧を考えたときの，系列上のラベルに対応する素性のタプルである．

２つのパスが値を共有している場合，そのパスは等価であるという．

素性構造に対して包含関係によって半順序関係を定義できる．

２つの素性構造について単一化が成功すれば両方を結合した情報を含む素性構造が作られる．

単一化によってパスが具体化されれば，等価なパスがすべて具体化される．（更新）

様々な言語的現象を簡単に解析するために素性構造を使うことができる．これには同氏の下位範疇化，転置構造，非有界依存構造，格による制約などが含まれる．